In [1]:
import socket
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.hashes import SHA256

# Function to decrypt data using the RSA private key
def decrypt_data(encrypted_data, private_key):
    try:
        decrypted_data = private_key.decrypt(
            encrypted_data,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        return decrypted_data
    except Exception as e:
        return f"Decryption failed: {str(e)}"

# Function to hash data using SHA-256
def hash_data(data):
    digest = hashes.Hash(SHA256())
    digest.update(data)
    return digest.finalize()

# Create an RSA key pair for the server
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

# Serialize the public key
public_key_pem = private_key.public_key().public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)

# Create a socket and bind it to a specific address and port
host = 'localhost'
port = 12339
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((host, port))

# Listen for incoming connections
server_socket.listen(1)
print("Server is listening...")

# Accept a connection from a client
client_socket, client_address = server_socket.accept()
print(f"Connection from {client_address}")

# Send the public key to the client
client_socket.send(public_key_pem)

# Receive the client's public key
client_public_key_pem = client_socket.recv(2048)

# Load the client's public key
client_public_key = serialization.load_pem_public_key(client_public_key_pem)

# Wait for the key exchange to complete
print("Key exchange completed. Waiting for encrypted message...")

# Receive the encrypted message from the client
encrypted_message = client_socket.recv(2048)
print(encrypted_message)
# Decrypt the message using the server's private key
decrypted_message = decrypt_data(encrypted_message, private_key)

# Split the message and hash
message, received_hash = decrypted_message.split(b'\n', 1)

# Calculate a new hash for the received message
calculated_hash = hash_data(message)

# Print the decrypted message
print(f"Decrypted Message: {message.decode('utf-8')}")

# Compare the received hash with the calculated hash
if received_hash == calculated_hash:
    print("Hashes match. Message integrity verified.")
else:
    print("Hashes do not match. Message integrity compromised.")

# Close the sockets
server_socket.close()
client_socket.close()


Server is listening...
Connection from ('127.0.0.1', 51662)
Key exchange completed. Waiting for encrypted message...
b"]\x81\x91\x9e\xd6\xb2i\x0b\xced\\\x15]\xbf;_\xa0O\xac\xfeNm\x06\xcb$'\xa9\tL\x0e%q\xdfE\xe3s:\x95\x8b\x12D\xa9t!\xc2O\x9f\xa3$\x1a`\x94\\\xa7a>%i\x85\x1f\xe4\xfb\xc2\xd8\x84\xcd,\xadA\x1c\xdd{H\xbb\x93\xc31\xb6\t{\xa3\x03KP$\x10\xe5>\xc4\xed\x87%\xd3A\x92\x9em\xe6\xa1\r_\x86\xb3&\x0b;\t>\xa4Ku\xbbW)tZ&H2\t\x99\xbdS\x81\xf3\xfc2\x985\\\xfa\xb2k\xad\xfb\x9f\xfa\x01\x0cOeB\xc5\x17\x94\xd0fl~K\x08\xca\x7f\xeaAP\xdf\xa4T\xf5\xe5[\xd5y\xd5\x03\x1b\xee\xe2\xb1\xe5;[\x81'&\xed\xbf{ \x96l\x04w\xa5\xb0\xb17\xe9B\x81g\xf5\t\xb8.%\x13\x1d\xda\x13\xd2A\x06\xf2\xb48M\x89\xdc\xfd\x05O\xa4\xc8u\xbe~`\x1dPl\x002[<)\xa1\xbf\xa0v\xac\x07\xc1\xcd\xb6\x07\xc6\xb5(\xf1\xf58\xe4\xd9U\xa5\xdd\xab\x97A\\\x16\x1a{\x81"
Decrypted Message: this is asymmetric
Hashes match. Message integrity verified.
